In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
pip install pymorphy2

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=3f91f3114cf01976e1e211886f377f1a55d1c931ba366f8ee5508b094c51c466
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [8]:
import pymorphy2
import gradio as gr
import re

morph = pymorphy2.MorphAnalyzer()

pos_translation = {
    'NOUN': 'іменник',
    'VERB': 'дієслово',
    'ADJF': 'прикметник',
    'INFN': 'інфінітив',
    'NUMR': 'числівник',
    'ADVB': 'прислівник',
    'NPRO': 'займенник',
    'PREP': 'прийменник',
    'CONJ': 'сполучник',
    'PRCL': 'частка',
    'INTJ': 'вигук',
}

gender_translation = {
    'masc': 'чоловічий',
    'femn': 'жіночий',
    'neut': 'середній',
}

number_translation = {
    'sing': 'однина',
    'plur': 'множина',
}

case_translation = {
    'nomn': 'називний',
    'gent': 'родовий',
    'datv': 'давальний',
    'accs': 'знахідний',
    'ablt': 'орудний',
    'loct': 'місцевий',
    'voct': 'кличний',
    'gen2': 'родовий (альтернативний)',
    'acc2': 'знахідний (альтернативний)',
    'loc2': 'місцевий (альтернативний)',
}

tense_translation = {
    'past': 'минулий',
    'pres': 'теперішній',
    'futr': 'майбутній',
}

def analyze_word(word):
    parsed_word = morph.parse(word)[0]
    pos = parsed_word.tag.POS
    info = {'Слово': word}

    if word in ['і', 'що', 'куди', 'звідки', 'при', 'на', 'з', 'в']:
        info['Частина мови'] = 'прийменник'
    elif word in ['ця', 'цей', 'це', 'цю', 'цій']:
        info['Частина мови'] = 'займенник'
    elif word.endswith(('ти', 'ть', 'ється', 'ться')) or pos == 'VERB':
        info['Частина мови'] = 'дієслово'
        info['Рід'] = gender_translation.get(parsed_word.tag.gender, 'невизначений')
        info['Число'] = number_translation.get(parsed_word.tag.number, 'невизначене')
        info['Відмінок'] = case_translation.get(parsed_word.tag.case, 'невизначений')
        info['Час'] = tense_translation.get(parsed_word.tag.tense, 'невизначений')
        info['Інфінітив'] = 'так' if 'INFN' in parsed_word.tag else 'ні'
    elif pos == 'NOUN' or word.endswith('у'):
        info['Частина мови'] = 'іменник'
        info['Рід'] = gender_translation.get(parsed_word.tag.gender, 'невизначений')
        info['Число'] = number_translation.get(parsed_word.tag.number, 'невизначене')
        info['Відмінок'] = case_translation.get(parsed_word.tag.case, 'невизначений')
    elif pos == 'ADJF' or word.endswith('і'):
        info['Частина мови'] = 'прикметник'
        info['Рід'] = gender_translation.get(parsed_word.tag.gender, 'невизначений')
        info['Відмінок'] = case_translation.get(parsed_word.tag.case, 'невизначений')
    elif pos == 'NUMR':
        info['Частина мови'] = 'числівник'
    elif pos == 'ADVB':
        info['Частина мови'] = 'прислівник'
    elif pos == 'NPRO':
        info['Частина мови'] = 'займенник'
    elif pos == 'PRCL':
        info['Частина мови'] = 'частка'
    elif pos == 'CONJ':
        info['Частина мови'] = 'сполучник'
    elif pos == 'INTJ':
        info['Частина мови'] = 'вигук'

    return {key: value for key, value in info.items() if value is not None}

def process_text(text):
    words = re.findall(r'\b\w+\b', text)
    results = [analyze_word(word) for word in words]
    formatted_result = '\n\n'.join(
        '{}:\n{}'.format(
            result['Слово'],
            '\n'.join(f'{k}: {v}' for k, v in result.items() if k != 'Слово')
        )
        for result in results
    )
    return formatted_result

with gr.Blocks() as demo:
    gr.Markdown('# Аналіз граматичної інформації')
    gr.Markdown('Цей модуль визначає граматичну інформацію слів українською мовою🤍.')
    with gr.Row():
        text_input = gr.Textbox(label='Введіть текст для аналізу🐼', placeholder='Напишіть слово або речення...', lines=4)
        text_output = gr.Textbox(label='Граматична інформація🐯', placeholder='Результат', lines=10)
    process_button = gr.Button('Обробити')
    process_button.click(fn=process_text, inputs=text_input, outputs=text_output)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://533a36f30fc3d8caa1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
